Now upto here we were providing a text and summarizing it using our llm but without any technique

tho there are many techniques to get the summary of any type of input

1) stuff document chain
2) Map Reduced
3) Refine

in stuff ,

we just load the document and combining all the given docs to get a text
and provide it to llm with some prompt
if is smaller one or fits in llm context size limits then it works but if not then

we have map reduced
(i) single prompt
(ii) multi-prompt

Map Reduced-->

               documents divided into chunks get summary of each chunk by providing a prompt

               and at last combining all summaries

               in multi prompt we just provide one more prompt before combining summaries

               docs->chunks->prompt to summarize each chunk -> final prompt to combine them

                and generate new summary(optional)->final summary

Refine--> here the process is same as Map reduced but for each chunk we provide summary of           the previous chunk (obv not for first chunk)

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GOOGLE_API_KEY"]=os.getenv("GEMINI_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite-preview-06-17")

Stuff Document Chain

Remember stuff just combines all documents into a combined text but we should split it so not to hit context limits

In [26]:
from langchain.document_loaders import PyPDFLoader
loader=PyPDFLoader("apjspeech.pdf")
loaded=loader.load_and_split()
for docs in loaded:
    print(docs)

page_content='A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have many unique experiences to share with you, which signify the 
following important messages: 
 
1. Accelerate development : Aspiration of the 

In [22]:
from langchain.prompts import PromptTemplate
template="Hii can you pls summarize this {text}"
prompt=PromptTemplate(input_variables=['text'],template=template)

In [23]:
from langchain.chains.summarize import load_summarize_chain
chain=load_summarize_chain(llm,chain_type='stuff',prompt=prompt,verbose=True)
chain.run(loaded)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Hii can you pls summarize this A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have many unique experi

'Dr. A.P.J. Abdul Kalam\'s departing speech is a heartfelt reflection on his five years as President of India, expressing gratitude to the people and sharing key messages derived from his interactions across the nation. He highlights ten core themes that emerged from his experiences:\n\n1.  **Accelerate Development through Youth Aspiration:** He emphasizes the powerful dream of a developed India, particularly as expressed by the nation\'s youth, and the need to channel this aspiration into national progress.\n2.  **Empower Villages:** Drawing from his visit to Nagaland, he stresses the importance of empowering villages with financial autonomy and better connectivity to facilitate their development and integration with urban areas.\n3.  **Mobilize Rural Core Competence for Competitiveness:** He showcases the success of the PURA (Providing Urban Amenities in Rural Areas) model, demonstrating how leveraging local skills and resources, coupled with technological support, can create globall

MAP REDUCED SUMMARIZER

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitted=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=100).split_documents(loaded)
splitted


[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

In [28]:
len(splitted)

13

In [30]:
chunks_prompt="""
Pls provide the summary for {text}
"""
map_prompt_template=PromptTemplate(input_variables=['text'],template=chunks_prompt)

In [31]:
final_prompt="""
Pls summarize this {text} and also keep in mind that
you have to give a motivation title ,summary in points and a powerful attracting introduction
"""
combine_prompt=PromptTemplate(input_variables=['text'],template=final_prompt)

In [36]:
output_chain=load_summarize_chain(llm,chain_type="map_reduce",map_prompt=map_prompt_template,combine_prompt=combine_prompt,verbose=True)
output_chain.run(splitted)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Pls provide the summary for A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have many unique expe

'Here\'s a summary of Dr. Kalam\'s messages, tailored for investors, with a motivational title, an attractive introduction, and points:\n\n## **Invest in India\'s Future: Cultivating Enduring Prosperity through Empowered Citizens**\n\n**Introduction:**\nImagine an investment landscape where the returns are not just financial, but deeply rooted in the tangible well-being and unleashed potential of an entire nation. Our vision is to build precisely this ecosystem – a developed India, driven by empowered youth, resilient villages, and a strong sense of national pride. This is more than a social aspiration; it\'s a strategic blueprint for a robust, stable, and exponentially growing economic environment, offering unparalleled opportunities for forward-thinking investors who believe in the power of collective progress.\n\n### Key Investment Pillars for a Developed India:\n\n*   **Harnessing Youth Power for Growth:** Investing in programs that **accelerate national development by engaging and

Refine Chain

In [33]:
chain=load_summarize_chain(llm,chain_type="refine",verbose=True)
chain.run(splitted)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have man

'In his departing speech, A.P.J. Abdul Kalam expressed profound gratitude for his five years in office, reflecting on the enriching experiences gained from interacting with diverse people and emphasizing India\'s potential to become a developed country well before 2020. He shared ten key messages derived from his interactions, highlighting the critical role of youth aspirations in driving accelerated development, the importance of rural empowerment, and the need for agricultural growth. Kalam stressed that problem-solving should be approached through partnership and courage, and that societal transformation can be achieved through connectivity. He also underscored the significance of national defense. Kalam firmly believed that the aspirations of young people for a prosperous, safe, and proud India should guide national efforts, noting that the desire to live in a developed India has deeply permeated the minds of the nation\'s youth.\n\nKalam illustrated rural empowerment with his visi